### Primero se importan las bibliotecas necesarias

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import time
import psutil
from collections import defaultdict
import seaborn as sns
# Especiales de RecSys
import surprise
from surprise import accuracy
from surprise import SVD
from surprise import SVDpp
from surprise.model_selection import PredefinedKFold
from surprise.model_selection import cross_validate

### Funciones utiles

In [2]:
def costo_computacional(algoritmo, trainset, imprimir=True):
    proceso = psutil.Process()
    inicio_tiempo = time.time()
    inicio_cpu = proceso.cpu_percent(interval=None)
    inicio_memoria = proceso.memory_info().rss

    algoritmo.fit(trainset)

    fin_tiempo = time.time()
    fin_cpu = proceso.cpu_percent(interval=None)
    fin_memoria = proceso.memory_info().rss

    tiempo = fin_tiempo - inicio_tiempo
    cpu = fin_cpu - inicio_cpu
    memoria = (fin_memoria - inicio_memoria) / (1024 * 1024)

    if imprimir:
        print(f"Tiempo entrenamiento: {tiempo:.2f} segundos")
        print(f"Uso de CPU: {cpu:.2f}%")
        print(f"Uso de memoria: {memoria:.2f} MB")
    return tiempo, cpu, memoria

def get_top_n(predictions, n=10):
    """Devuelve las N-mejores recomendaciones para cada usuario de un set de predicción.

    Args:
        predictions(lista de objetos Prediction): La lista de predicción obtenida del método test.
        n(int): El número de recomendaciónes por usuario

    Returns:
    Un diccionario donde las llaves son ids de usuario y los valores son listas de tuplas:
        [(item id, rating estimation), ...] de tamaño n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

### Datos a utilizar

In [3]:
train_data = pd.read_csv("datos/train.csv")
validation_data = pd.read_csv("datos/validation.csv")

In [4]:
validation_data.head()

,rating,review_title,text,timestamp,item_title,item_id,user_id
0,5.0,Five Stars,yay,2015-03-16 16:15:22.000,WWF Smackdown 2: Know Your Role,23386,238159
1,5.0,Fantastic Reprise of a Classic,Shogun: Total War was the title that marked th...,2011-05-11 14:40:55.000,Total War: Shogun 2 -The Blood Pack DLC [Onlin...,11444,9040
2,3.0,Staircase movement action sucks,Remember block graphics? Like when Fry become...,2009-04-02 14:38:37.000,Final Fantasy Crystal Chronicles: Echoes of Ti...,14619,502328
3,3.0,Three Stars,nice!,2015-06-08 16:52:08.000,Age of Empires III: The Asian Dynasties Expans...,11644,210635
4,5.0,Just what I needed!,It came on time and in perfect condition. I've...,2013-01-18 08:21:26.000,Xbox 360 Data Transfer Cable,17020,1665901


### 1. Random

In [5]:
# Creamos un database de predicciones aleatorias que puede ser utilizado por get_top_n
random_pred = validation_data[["user_id", "item_id", "rating"]].copy()
random_pred["estimado"] = np.random.uniform(1, 5, size=len(random_pred))

# Se convierte en una lista de tuplas en el formato (uid, iid, true_r, est, _)
predictions_random = []
for _, row in random_pred.iterrows():
    predictions_random.append((int(row['user_id']), int(row['item_id']), float(row['rating']), float(row['estimado']), None))

# Obtener las mejores N recomendaciones
top_n_random = get_top_n(predictions_random, n=10)

### 2. Item-based collaborative filtering

In [6]:
# La lectura de datos sirve tanto para item-based como para funkSVD al usar Surprise
reader = surprise.Reader(rating_scale=(1,5))
trainset = surprise.Dataset.load_from_df(train_data[["user_id", "item_id", "rating"]], reader).build_full_trainset()
testset = list(zip(validation_data["user_id"], validation_data["item_id"], validation_data["rating"]))

# Parte especifica de entrenamiento del algoritmo
myItemKnn = surprise.KNNBasic(k=7, sim_options={'name': 'pearson', 'user_based': False})
tiempo, cpu, memoria =costo_computacional(myItemKnn, trainset, False)
predictions_iknn = myItemKnn.test(testset)

# Obtener las mejores N recomendaciones
top_n_iknn = get_top_n(predictions_iknn, n=10)

Computing the pearson similarity matrix...
Done computing similarity matrix.


### 3. FunkSVD

In [7]:
# Parte especifica de entrenamiento del algoritmo
funksvd = SVD(n_factors = 100, n_epochs = 20, lr_all = 0.005, reg_all = 0.02)
tiempo, cpu, memoria =costo_computacional(funksvd, trainset, False)
predictions_funksvd = funksvd.test(testset)

# Obtener las mejores N recomendaciones
top_n_funksvd = get_top_n(predictions_funksvd, n=10)

### 4. ALS

### 5. BPR

### 6. Factorization Machines